# S1 Coursework: Photon Detector Calibration Analysis

*Your Name (crsid@cam.ac.uk)*

## Problem Statement

This analysis determines the performance parameters of a photon detector using calibration data. The detector measures energy $E$ when photons of known energy $E_0$ are incident. We aim to estimate 5 parameters:

**Mean energy model:**
$$\mu_E = \lambda E_0 + \Delta$$

**Width (resolution) model:**
$$\left( \frac{\sigma_E}{E_0} \right)^2 = \left( \frac{a}{\sqrt{E_0}} \right)^2 + \left( \frac{b}{E_0} \right)^2 + c^2$$

We analyze calibration data at 7 energy points ($E_0$ = 20, 30, 40, 50, 60, 70, 80 GeV) with 1000 measurements each.

---
## Setup

### Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Import s1_sol package modules
from s1_sol.data_loader import load_sample_data, get_residuals, get_residuals_by_energy
from s1_sol.statistics import compute_sample_stats  # You'll need to implement fit functions
from s1_sol.models import mean_energy_model, width_energy_model, relative_width_squared
from s1_sol.fitting import fit_normal_mle, fit_individual_energies, fit_simultaneous_all_energies
from s1_sol.plotting import setup_plot_style, plot_residuals_total, plot_residuals_by_energy
from s1_sol.bootstrap import bootstrap_sample, run_bootstrap_analysis, compute_bootstrap_confidence_interval
from s1_sol.results_manager import ResultsManager

# Set up plot styling
setup_plot_style()

# Create figs directory if it doesn't exist
os.makedirs('../figs', exist_ok=True)

# Initialize results manager
results = ResultsManager()

print("All imports successful!")

### Load Data

In [ ]:
# Load the calibration sample
df, data_dict, e0_values = load_sample_data('../sample.csv')

print(f"Loaded {len(df)} total measurements")
print(f"Energy points: {e0_values}")
print(f"\nMeasurements per energy point:")
for e0 in e0_values:
    print(f"  E0 = {e0} GeV: {len(data_dict[e0])} measurements")

# Display first few rows
df.head(10)

---
## 1. Plotting Sample Estimates

### 1(i) Plot Total Sample

We first visualize all energy residuals ($E_{rec} - E_{true}$) combined.

In [ ]:
fig, ax = plot_residuals_total(df)
fig.savefig('../figs/Figure1.1.pdf')
plt.show()

print("Figure 1.1 saved.")

**Observation:** [Add your interpretation here]

### 1(ii) Overlay Samples at Each $E_0$

Now we overlay the residual distributions for each true energy value to see how they differ.

In [ ]:
fig, ax = plot_residuals_by_energy(data_dict)
fig.savefig('../figs/Figure1.2.pdf')
plt.show()

print("Figure 1.2 saved.")

**Observation:** [Add your interpretation - do widths change with energy?]

### 1(iii) Sample Estimates

Compute sample mean $\hat{\mu}_{\text{samp}}$ and standard deviation $\hat{\sigma}_{\text{samp}}$ at each $E_0$, along with their standard errors.

In [ ]:
# Compute sample statistics
stats_df = compute_sample_stats(data_dict)

print("Sample Statistics:")
print(stats_df.to_string(index=False))

Now plot the sample mean and standard deviation as a function of $E_0$:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12.8, 4.8))

# Left plot: Sample means vs E0
ax[0].errorbar(stats_df['E0'], stats_df['mean'], yerr=stats_df['mean_err'],
               fmt='o', capsize=3, label='Sample mean')
ax[0].set_xlabel('$E_0$ [GeV]')
ax[0].set_ylabel('$\\hat{\\mu}_{\\text{samp}}$ [GeV]')
ax[0].set_title('Sample Mean vs True Energy')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Right plot: Sample standard deviations vs E0
ax[1].errorbar(stats_df['E0'], stats_df['std'], yerr=stats_df['std_err'],
               fmt='o', capsize=3, label='Sample std', color='C1')
ax[1].set_xlabel('$E_0$ [GeV]')
ax[1].set_ylabel('$\\hat{\\sigma}_{\\text{samp}}$ [GeV]')
ax[1].set_title('Sample Std Dev vs True Energy')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('../figs/Figure1.3.pdf')
plt.show()

print("Figure 1.3 saved.")

**Observation:** [Comment on trends - is mean linear? How does width change?]

### 1(iv) Fit Trends

Perform weighted least squares fits to estimate parameters $\{\lambda, \Delta, a, b, c\}$.

**TODO:** You need to implement `fit_mean_model_least_squares` and `fit_width_model_least_squares` in `s1_sol/statistics.py`

In [ ]:
# Import the least squares functions
from s1_sol.statistics import fit_mean_model_least_squares, fit_width_model_least_squares

# Fit mean model
lambda_samp, delta_samp, lambda_err_samp, delta_err_samp = fit_mean_model_least_squares(
    stats_df['E0'].values, stats_df['mean'].values, stats_df['mean_err'].values
)

# Fit width model
a_samp, b_samp, c_samp, a_err_samp, b_err_samp, c_err_samp = fit_width_model_least_squares(
    stats_df['E0'].values, stats_df['std'].values, stats_df['std_err'].values
)

print(f"Sample Estimates from Least Squares:")
print(f"λ = {lambda_samp:.4f} ± {lambda_err_samp:.4f}")
print(f"Δ = {delta_samp:.4f} ± {delta_err_samp:.4f}")
print(f"a = {a_samp:.4f} ± {a_err_samp:.4f}")
print(f"b = {b_samp:.4f} ± {b_err_samp:.4f}")
print(f"c = {c_samp:.4f} ± {c_err_samp:.4f}")

# Save to results
results.update('sample_ests',
               {'lb': lambda_samp, 'dE': delta_samp, 'a': a_samp, 'b': b_samp, 'c': c_samp},
               {'lb': lambda_err_samp, 'dE': delta_err_samp, 'a': a_err_samp, 'b': b_err_samp, 'c': c_err_samp})

Plot the fitted trends with rescaled axes ($\hat{\mu}_{\text{samp}} - E_0$ and $\hat{\sigma}_{\text{samp}} / E_0$):

In [ ]:
# Create Figure 1.4 with fitted curves
# Note: Bootstrap error bands will be added in Section 4

fig, ax = plt.subplots(1, 2, figsize=(12.8, 4.8))

# Create fine grid for smooth curves
e0_smooth = np.linspace(e0_values.min(), e0_values.max(), 100)

# Left plot: μ - E0 (rescaled mean)
rescaled_mean = stats_df['mean'].values - stats_df['E0'].values
rescaled_mean_fit = mean_energy_model(e0_smooth, lambda_samp, delta_samp) - e0_smooth

ax[0].errorbar(stats_df['E0'], rescaled_mean, yerr=stats_df['mean_err'],
               fmt='ko', label='Sample estimates', capsize=3)
ax[0].plot(e0_smooth, rescaled_mean_fit, 'r-', linewidth=2,
           label=f'Fit: λ={lambda_samp:.4f}, Δ={delta_samp:.4f}')
ax[0].set_xlabel('$E_0$ [GeV]')
ax[0].set_ylabel('$\\hat{\\mu}_{\\text{samp}} - E_0$ [GeV]')
ax[0].set_title('Rescaled Mean vs True Energy')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Right plot: σ/E0 (relative width)
relative_width = stats_df['std'].values / stats_df['E0'].values
relative_width_err = stats_df['std_err'].values / stats_df['E0'].values
relative_width_fit = width_energy_model(e0_smooth, a_samp, b_samp, c_samp) / e0_smooth

ax[1].errorbar(stats_df['E0'], relative_width, yerr=relative_width_err,
               fmt='ko', label='Sample estimates', capsize=3)
ax[1].plot(e0_smooth, relative_width_fit, 'b-', linewidth=2,
           label=f'Fit: a={a_samp:.4f}, b={b_samp:.4f}, c={c_samp:.4f}')
ax[1].set_xlabel('$E_0$ [GeV]')
ax[1].set_ylabel('$\\hat{\\sigma}_{\\text{samp}} / E_0$')
ax[1].set_title('Relative Width vs True Energy')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

fig.tight_layout()
fig.savefig('../figs/Figure1.4.pdf')
plt.show()

print("Figure 1.4 saved.")

---
## 2. Individual Fits

### 2(i) Normal Fits

Perform unbinned maximum likelihood fits using a normal distribution at each $E_0$ independently.

In [ ]:
# Fit normal distribution at each energy point
indiv_fits_df = fit_individual_energies(data_dict)

print("Individual MLE Fits:")
print(indiv_fits_df.to_string(index=False))

Create plots showing the data with fitted normal distributions:

In [ ]:
# TODO: Create Figure 2.1 showing individual fits
# Left: Overlaid histograms with fitted curves
# Right: Total histogram with sum of fitted distributions

fig, ax = plt.subplots(1, 2, figsize=(12.8, 4.8))

# TODO: Add your plotting code here

fig.savefig('../figs/Figure2.1.pdf')
plt.show()

print("Figure 2.1 saved.")

### 2(ii) Fit Trends

Use the fitted means and widths from individual MLE fits to determine $\{\lambda, \Delta, a, b, c\}$ via least squares.

In [ ]:
# Use the least squares functions on the individual fit results

lambda_indiv, delta_indiv, lambda_err_indiv, delta_err_indiv = fit_mean_model_least_squares(
    indiv_fits_df['E0'].values, indiv_fits_df['mu'].values, indiv_fits_df['mu_err'].values
)

a_indiv, b_indiv, c_indiv, a_err_indiv, b_err_indiv, c_err_indiv = fit_width_model_least_squares(
    indiv_fits_df['E0'].values, indiv_fits_df['sigma'].values, indiv_fits_df['sigma_err'].values
)

print(f"Individual Fits + Least Squares:")
print(f"λ = {lambda_indiv:.4f} ± {lambda_err_indiv:.4f}")
print(f"Δ = {delta_indiv:.4f} ± {delta_err_indiv:.4f}")
print(f"a = {a_indiv:.4f} ± {a_err_indiv:.4f}")
print(f"b = {b_indiv:.4f} ± {b_err_indiv:.4f}")
print(f"c = {c_indiv:.4f} ± {c_err_indiv:.4f}")

# Save to results
results.update('individual_fits',
               {'lb': lambda_indiv, 'dE': delta_indiv, 'a': a_indiv, 'b': b_indiv, 'c': c_indiv},
               {'lb': lambda_err_indiv, 'dE': delta_err_indiv, 'a': a_err_indiv, 'b': b_err_indiv, 'c': c_err_indiv})

In [ ]:
# TODO: Create Figure 2.2 - fitted trends with bootstrap error bands

fig, ax = plt.subplots(1, 2, figsize=(12.8, 4.8))

# TODO: Add your plotting code here

fig.savefig('../figs/Figure2.2.pdf')
plt.show()

print("Figure 2.2 saved.")

---
## 3. Simultaneous Fit

### 3(i) The Simultaneous Likelihood

The simultaneous likelihood fits all 5 parameters directly to the full dataset:

$$\mathcal{L}(\lambda, \Delta, a, b, c) = \prod_{i=1}^{7} \prod_{j=1}^{1000} \frac{1}{\sqrt{2\pi}\sigma_E(E_{0,i})} \exp\left[-\frac{(E_{ij} - \mu_E(E_{0,i}))^2}{2\sigma_E^2(E_{0,i})}\right]$$

where $\mu_E(E_0) = \lambda E_0 + \Delta$ and $\sigma_E(E_0) = E_0 \sqrt{(a/\sqrt{E_0})^2 + (b/E_0)^2 + c^2}$.

In [ ]:
# Perform simultaneous fit
params_simul, errors_simul, minuit_obj = fit_simultaneous_all_energies(data_dict)

print("Simultaneous Fit Results:")
print(f"λ = {params_simul['lam']:.4f} ± {errors_simul['lam']:.4f}")
print(f"Δ = {params_simul['delta']:.4f} ± {errors_simul['delta']:.4f}")
print(f"a = {params_simul['a']:.4f} ± {errors_simul['a']:.4f}")
print(f"b = {params_simul['b']:.4f} ± {errors_simul['b']:.4f}")
print(f"c = {params_simul['c']:.4f} ± {errors_simul['c']:.4f}")
print(f"\nFit valid: {minuit_obj.valid}")
print(f"Fit accurate: {minuit_obj.accurate}")

In [ ]:
# TODO: Create Figure 3.1 - fitted curves with bootstrap error bands

fig, ax = plt.subplots(1, 2, figsize=(12.8, 4.8))

# TODO: Add your plotting code here

fig.savefig('../figs/Figure3.1.pdf')
plt.show()

print("Figure 3.1 saved.")

### 3(ii) Saving the Results

In [ ]:
# Update results with simultaneous fit
results.update('simultaneous_fit',
               {'lb': params_simul['lam'], 'dE': params_simul['delta'],
                'a': params_simul['a'], 'b': params_simul['b'], 'c': params_simul['c']},
               {'lb': errors_simul['lam'], 'dE': errors_simul['delta'],
                'a': errors_simul['a'], 'b': errors_simul['b'], 'c': errors_simul['c']})

# Save results.json
results.save('../results.json')

# Display results
results.display()

### 3(iii) Comparing the Results

Compare parameter estimates from all three methods.

In [ ]:
# TODO: Create Figure 3.2 - comparison plot of all three methods

fig, ax = plt.subplots(figsize=(6.4, 6.4))

# TODO: Add your comparison plot here

fig.savefig('../figs/Figure3.2.pdf')
plt.show()

print("Figure 3.2 saved.")

---
## 4. Bootstrap Entire Sample

Run non-parametric bootstrap with 2500 samples to validate uncertainty estimates.

In [ ]:
# TODO: Implement bootstrap analysis
# This will take ~1 minute

# def analysis_function(bootstrap_data):
#     # Run all three methods on bootstrap sample
#     # Return dict with all parameters
#     pass

# bootstrap_results = run_bootstrap_analysis(data_dict, analysis_function, n_bootstrap=2500)

print("TODO: Implement bootstrap analysis")

In [ ]:
# TODO: Create Figure 4.1 - Bootstrap distributions (5 subplots, one per parameter)

fig, ax = plt.subplots(2, 3, figsize=(19.2, 9.6))

# TODO: Add bootstrap histograms

ax[0,0].set_xlabel("$\\lambda$")
ax[0,1].set_xlabel("$\\Delta$")
ax[0,2].set_visible(False)
ax[1,0].set_xlabel("$a$")
ax[1,1].set_xlabel("$b$")
ax[1,2].set_xlabel("$c$")

fig.savefig('../figs/Figure4.1.pdf')
plt.show()

print("Figure 4.1 saved.")

In [ ]:
# TODO: Create Figure 4.2 - Comparison with bootstrap uncertainties

fig, ax = plt.subplots(figsize=(6.4, 6.4))

# TODO: Add comparison plot with both original and bootstrap uncertainties

fig.savefig('../figs/Figure4.2.pdf')
plt.show()

print("Figure 4.2 saved.")

---
## 5. Discussion of Results

*Maximum 500 words*

TODO: Add your discussion here. Consider:
- Do the three methods (sample estimates, individual fits, simultaneous fit) agree?
- Which method gives the most precise estimates? Why?
- How do the bootstrap uncertainties compare to the fitted uncertainties?
- Are the parameter values physically reasonable?
- What do the fitted parameters tell us about detector performance?
- Would you redesign the analysis in any way?
- Any unexpected findings or discrepancies?